AQUI SE HARAN PREDICCIONES CON EL MODELO DE PROPHET ENTRENADO

In [119]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
import numpy as np
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle
from prophet.plot import plot_plotly, plot_components_plotly
import plotly.express as px
# CONFIGURAMOS QUE LOS GRAFICOS SE ABRAN EN WEB PARA PODER VERLOS INTERACTIVOS
import plotly.io as pio
pio.renderers.default = 'browser'

In [ ]:
def prophet_model():
    df = pd.read_csv('../data/demanda-depurado.csv')

    df['fecha'] = pd.to_datetime(df['fecha'])
    df = df.sort_values('fecha')

    # LISTA VACIA PARA AÑADIR LOS VALORES DE LAS PREDICCIONES POR REGION.
    predicciones = []

    for (region, indicador), grupo in df.groupby(['region', 'indicador']):
        # FILTRAR POR REGION E INDICADOR PARA HACER UNO A LA VEZ.
        df_filtrado = grupo[['fecha', 'valor']].rename(columns={'fecha': 'ds', 'valor': 'y'})

        # PREPARACIÓN DE LOS DATOS.
        df_filtrado = df_filtrado.dropna()
        df_filtrado = df_filtrado[np.isfinite(df_filtrado['y'])]

        # HAREMOS UNA LIMPIEZA DE DATOS ALGO AGRESIVA DEBIDO A
        # LA GRAN MAGNITUD DE VALORES QUE SE MANEJAN.
        z_scores = zscore(df_filtrado['y'])
        df_filtrado = df_filtrado[np.abs(z_scores) < 3]

        lower_bound = np.percentile(df_filtrado['y'], 1)
        upper_bound = np.percentile(df_filtrado['y'], 99)
        df_filtrado = df_filtrado[(df_filtrado['y'] >= lower_bound) & (df_filtrado['y'] <= upper_bound)]

        # ESCALAMOS LOS DATOS DEBIDO A QUE TIENEN VALORES MUY GRANDES
        # Y ASI EVITAMOS QUE EL MODELO TENGA UN COLAPSO INTERNO.
        df_filtrado['y'] = np.log(df_filtrado['y'] + 1)

        # CONFIGURACIÓN DEL MODELO PROPHET DE META PARA HACER LAS PREDICCIONES.
        modelo = Prophet(yearly_seasonality=False, daily_seasonality=True, weekly_seasonality=True)
        modelo.fit(df_filtrado)
        nombre_archivo = f"modelo_prophet_{region}.pkl"
        with open(f'../../../models/prophet_models/{nombre_archivo}', 'wb') as f:
            pickle.dump(modelo, f)

        # DEFINIR EL RANGO DE FECHAS A PREDECIR.
        rango = 24
        future = modelo.make_future_dataframe(periods=rango, freq='D')

        forecast = modelo.predict(future)

        # GRAFICAS PARA VISUALIZAR LAS PREDICCIONES.
        fig1 = plot_plotly(modelo, forecast)
        fig1.update_layout(
            title=f'Predicción de demanda para {region} usando Prophet',
            xaxis_title='Fecha',
            yaxis_title='Demanda'
        )

        fig2 = plot_components_plotly(modelo, forecast)
        fig2.update_layout(
            title=f'Componentes de la predicción de demanda para {region}',
            xaxis_title='Fecha',
            yaxis_title='Demanda'
        )

        fig1.show()
        fig2.show()

        # HACEMOS LAS PREDICCIONES PARA EL RANGO DE FECHAS SELECCIONADO Y LAS AÑADIMOS A UNA LISTA
        # PARA PODER VISUALIZARLAS DESPUES.
        df_futuro = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(rango)
        predicciones.append(df_futuro)

    df_predicciones_totales = pd.concat(predicciones, ignore_index=True)

    return df_predicciones_totales

In [123]:
prophet = prophet_model()

01:03:40 - cmdstanpy - INFO - Chain [1] start processing
01:03:40 - cmdstanpy - INFO - Chain [1] done processing
01:03:41 - cmdstanpy - INFO - Chain [1] start processing
01:03:41 - cmdstanpy - INFO - Chain [1] done processing
01:03:41 - cmdstanpy - INFO - Chain [1] start processing
01:03:41 - cmdstanpy - INFO - Chain [1] done processing
01:03:42 - cmdstanpy - INFO - Chain [1] start processing
01:03:42 - cmdstanpy - INFO - Chain [1] done processing
01:03:42 - cmdstanpy - INFO - Chain [1] start processing
01:03:42 - cmdstanpy - INFO - Chain [1] done processing
01:03:43 - cmdstanpy - INFO - Chain [1] start processing
01:03:43 - cmdstanpy - INFO - Chain [1] done processing
01:03:43 - cmdstanpy - INFO - Chain [1] start processing
01:03:43 - cmdstanpy - INFO - Chain [1] done processing
01:03:44 - cmdstanpy - INFO - Chain [1] start processing
01:03:44 - cmdstanpy - INFO - Chain [1] done processing
01:03:44 - cmdstanpy - INFO - Chain [1] start processing
01:03:44 - cmdstanpy - INFO - Chain [1]